In [2]:
from manim import *

class NepaliTrigTable(Scene):
    def construct(self):
        # --- CONFIGURATION ---
        nepali_font = "Mangal"  # Change to "Noto Sans Devanagari" if on Linux/Colab
        
        # Reduced spacing to fit screen perfectly
        h_buff = 1.8  # Width of columns
        v_buff = 0.75 # Height of rows
        
        # Shift table UP to leave space at the bottom for calculations
        start_pos = UP * 2.8 + LEFT * 4.5
        
        # Scratchpad location (Calculation Zone)
        scratchpad_pos = DOWN * 2.5
        
        # --- TITLE ---
        title = Text("त्रिकोणमितीय तालिका (Trigonometric Table)", font=nepali_font, font_size=32)
        title.to_edge(UP, buff=0.1)
        self.play(Write(title))

        # --- GRID CREATION ---
        rows = 7 # Header + 6 functions
        cols = 6 # Header + 5 angles
        
        grid = VGroup()
        # Horizontal lines
        for i in range(rows + 1):
            line = Line(
                start_pos + DOWN * i * v_buff, 
                start_pos + DOWN * i * v_buff + RIGHT * cols * h_buff, 
                stroke_width=1.5, color=GRAY
            )
            grid.add(line)
        # Vertical lines
        for i in range(cols + 1):
            line = Line(
                start_pos + RIGHT * i * h_buff, 
                start_pos + RIGHT * i * h_buff + DOWN * rows * v_buff, 
                stroke_width=1.5, color=GRAY
            )
            grid.add(line)

        self.play(Create(grid), run_time=1.0)

        # --- HEADERS ---
        angles = [r"0^\circ", r"30^\circ", r"45^\circ", r"60^\circ", r"90^\circ"]
        funcs = [r"\sin\theta", r"\cos\theta", r"\tan\theta", r"\csc\theta", r"\sec\theta", r"\cot\theta"]
        
        # Helper for cell centers
        def get_cell(r, c):
            return start_pos + RIGHT * c * h_buff + RIGHT * (h_buff/2) + \
                   DOWN * r * v_buff + DOWN * (v_buff/2)

        # Draw Angle Headers (Row 0)
        self.add(MathTex(r"\theta", font_size=28).move_to(get_cell(0, 0)))
        for i, angle in enumerate(angles):
            self.add(MathTex(angle, font_size=28).move_to(get_cell(0, i+1)))

        # Draw Function Headers (Col 0)
        for i, func in enumerate(funcs):
            self.add(MathTex(func, font_size=28).move_to(get_cell(i+1, 0)))

        # --- DATA STORAGE ---
        # 0:Sin, 1:Cos, 2:Tan, 3:Csc, 4:Sec, 5:Cot
        # We store the Mobjects here to reference them later
        table_mobs = [[None for _ in range(5)] for _ in range(6)]
        
        # Raw LaTeX values
        sin_vals = ["0", r"\frac{1}{2}", r"\frac{1}{\sqrt{2}}", r"\frac{\sqrt{3}}{2}", "1"]
        cos_vals = ["1", r"\frac{\sqrt{3}}{2}", r"\frac{1}{\sqrt{2}}", r"\frac{1}{2}", "0"]
        tan_vals = ["0", r"\frac{1}{\sqrt{3}}", "1", r"\sqrt{3}", r"\infty"]
        
        # --- 1. SINE (Standard Fill) ---
        text_sin = Text("sin को मानहरू (Standard Values)", font=nepali_font, font_size=24, color=YELLOW).move_to(scratchpad_pos)
        self.play(Write(text_sin))
        
        for i, val in enumerate(sin_vals):
            mob = MathTex(val, font_size=28).move_to(get_cell(1, i+1))
            table_mobs[0][i] = mob
            self.play(FadeIn(mob, shift=UP * 0.2), run_time=0.2)
        
        self.wait(0.5)
        self.play(FadeOut(text_sin))

        # --- 2. COSINE (Derivation via Copy & Transform) ---
        # Formula Display
        formula_cos = MathTex(r"\cos\theta = \sin(90^\circ - \theta)", color=BLUE, font_size=30).move_to(scratchpad_pos + UP * 0.5)
        desc_cos = Text("पूरक कोण (Complementary Angle) बाट निकालौँ", font=nepali_font, font_size=20, color=BLUE).next_to(formula_cos, DOWN)
        self.play(Write(formula_cos), Write(desc_cos))
        self.wait(1)

        for i in range(5):
            # 1. Identify Source (Sin) and Target (Cos)
            # Cos(0) needs Sin(90-0) -> Sin(90)
            target_angle_idx = 4 - i 
            source_mob = table_mobs[0][target_angle_idx] # The Sin value
            
            # 2. Create a Copy and move to Scratchpad
            # We explicitly show "Sin(90-x) -> Value" logic
            temp_copy = source_mob.copy()
            temp_copy.set_color(BLUE)
            
            # Move to scratchpad area to show "processing"
            calculation_spot = scratchpad_pos + DOWN * 0.5
            self.play(temp_copy.animate.move_to(calculation_spot), run_time=0.5)
            
            # 3. Create the final value at scratchpad
            final_val_tex = cos_vals[i]
            final_mob = MathTex(final_val_tex, font_size=28, color=WHITE).move_to(calculation_spot)
            
            # Transform Copy -> Final (Visual confirmation)
            # (In this case Sin value is same as Cos value, but the visual 'landing' helps)
            self.play(Transform(temp_copy, final_mob), run_time=0.3)
            
            # 4. Move Final Result to Table
            target_cell = get_cell(2, i+1)
            self.play(temp_copy.animate.move_to(target_cell), run_time=0.5)
            
            # Store and cleanup
            table_mobs[1][i] = temp_copy

        self.play(FadeOut(formula_cos), FadeOut(desc_cos))

        # --- 3. TANGENT (Calculation Fraction) ---
        formula_tan = MathTex(r"\tan\theta = \frac{\sin\theta}{\cos\theta}", color=GREEN, font_size=30).move_to(scratchpad_pos + UP * 0.5)
        desc_tan = Text("भाग गरेर निकालौँ (Division)", font=nepali_font, font_size=20, color=GREEN).next_to(formula_tan, DOWN)
        self.play(Write(formula_tan), Write(desc_tan))
        
        for i in range(5):
            sin_mob = table_mobs[0][i]
            cos_mob = table_mobs[1][i]
            
            # 1. Copy Sin and Cos to scratchpad
            s_copy = sin_mob.copy().set_color(RED)
            c_copy = cos_mob.copy().set_color(RED)
            
            # Arrange them as a fraction in the scratchpad
            # Sin on top, Cos on bottom
            calc_center = scratchpad_pos + DOWN * 0.5
            fraction_line = Line(LEFT*0.3, RIGHT*0.3).move_to(calc_center)
            s_copy.next_to(fraction_line, UP, buff=0.1)
            c_copy.next_to(fraction_line, DOWN, buff=0.1)
            
            group_frac = VGroup(s_copy, fraction_line, c_copy)
            
            # Animate the copy coming down
            self.play(TransformFromCopy(sin_mob, s_copy), TransformFromCopy(cos_mob, c_copy), Create(fraction_line), run_time=0.5)
            
            # 2. Transform the fraction into the result
            result_mob = MathTex(tan_vals[i], font_size=28).move_to(calc_center)
            
            self.play(Transform(group_frac, result_mob), run_time=0.6)
            self.wait(0.2)
            
            # 3. Move result to table
            target_cell = get_cell(3, i+1)
            self.play(group_frac.animate.move_to(target_cell), run_time=0.5)
            
            table_mobs[2][i] = group_frac # Store the final moved object

        self.play(FadeOut(formula_tan), FadeOut(desc_tan))

        # --- 4. RECIPROCALS (Quick Fill to save time) ---
        # We will do these faster but still show the relationship
        reciprocal_text = Text("बाँकी अनुपातहरू उल्टो (Reciprocal) हुन्", font=nepali_font, font_size=24, color=ORANGE).move_to(scratchpad_pos)
        self.play(Write(reciprocal_text))

        # Definitions
        csc_vals = [r"\infty", "2", r"\sqrt{2}", r"\frac{2}{\sqrt{3}}", "1"]
        sec_vals = ["1", r"\frac{2}{\sqrt{3}}", r"\sqrt{2}", "2", r"\infty"]
        cot_vals = [r"\infty", r"\sqrt{3}", "1", r"\frac{1}{\sqrt{3}}", "0"]
        
        # Fill remaining rows row-by-row
        # Cosec (from Sin)
        for i, val in enumerate(csc_vals):
            # Highlight parent
            self.play(Indicate(table_mobs[0][i], color=RED, scale_factor=1.2), run_time=0.2)
            mob = MathTex(val, font_size=28).move_to(get_cell(4, i+1))
            self.add(mob)
        
        # Sec (from Cos)
        for i, val in enumerate(sec_vals):
            self.play(Indicate(table_mobs[1][i], color=ORANGE, scale_factor=1.2), run_time=0.2)
            mob = MathTex(val, font_size=28).move_to(get_cell(5, i+1))
            self.add(mob)

        # Cot (from Tan)
        for i, val in enumerate(cot_vals):
            self.play(Indicate(table_mobs[2][i], color=PURPLE, scale_factor=1.2), run_time=0.2)
            mob = MathTex(val, font_size=28).move_to(get_cell(6, i+1))
            self.add(mob)

        self.play(FadeOut(reciprocal_text))
        
        final_msg = Text("तालिका पूरा भयो", font=nepali_font, color=YELLOW).move_to(scratchpad_pos)
        self.play(Write(final_msg))
        self.wait(2)

%manim -qk -v error NepaliTrigTable

Manim Community v0.19.1